In [22]:
import torch
import jovian
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

In [23]:
DATA_FILENAME = "car data.csv"
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()


,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [24]:
your_name = "Viththakan Ragupathi" # at least 5 characters
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.Year = dataframe.Year * ord(rand_str[1])/100.
    # scale target
    dataframe.Selling_Price = dataframe.Selling_Price * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['Car_Name'], axis=1)
    return dataframe

dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
283,city,2116.80,10.4284,11.80,9010,Petrol,Dealer,Manual,0
50,fortuner,2112.60,17.2840,30.61,104707,Diesel,Dealer,Automatic,0
261,brio,2116.80,6.3800,5.97,5600,Petrol,Dealer,Manual,0
51,fortuner,2115.75,26.6800,30.61,40000,Diesel,Dealer,Automatic,0
43,dzire,2114.70,6.3800,8.06,45780,Diesel,Dealer,Manual,0


In [25]:
input_cols = ["Year","Present_Price","Kms_Driven","Owner"]
categorical_cols = ["Fuel_Type","Seller_Type","Transmission"]
output_cols = ["Selling_Price"]

In [26]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[2.11680e+03, 1.18000e+01, 9.01000e+03, 0.00000e+00],
        [2.11260e+03, 3.06100e+01, 1.04707e+05, 0.00000e+00],
        [2.11680e+03, 5.97000e+00, 5.60000e+03, 0.00000e+00],
        ...,
        [2.11785e+03, 1.47000e+00, 1.10000e+04, 0.00000e+00],
        [2.11680e+03, 5.50000e-01, 3.10000e+04, 0.00000e+00],
        [2.11680e+03, 5.70000e-01, 2.40000e+04, 0.00000e+00]]),
 array([[10.4284],
        [17.284 ],
        [ 6.38  ],
        [26.68  ],
        [ 6.38  ],
        [ 4.756 ],
        [13.282 ],
        [ 6.09  ],
        [ 0.4872],
        [11.02  ],
        [ 3.596 ],
        [ 6.148 ],
        [ 0.522 ],
        [ 0.8352],
        [ 1.74  ],
        [ 8.7   ],
        [ 3.306 ],
        [ 4.002 ],
        [ 7.54  ],
        [ 0.29  ],
        [ 1.914 ],
        [ 0.348 ],
        [ 5.22  ],
        [ 0.232 ],
        [ 1.16  ],
        [ 5.742 ],
        [ 2.958 ],
        [11.252 ],
        [ 0.1856],
        [19.72  ],
        [ 4.814 ],
        [ 5.51  ],
     

In [27]:
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

dataset = TensorDataset(inputs, targets)
train_ds, val_ds = random_split(dataset, [228, 57])
batch_size = 128

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [28]:
input_size = len(input_cols)
output_size = len(output_cols)

class CarsModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)                         # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))
            
model = CarsModel()

list(model.parameters())


[Parameter containing:
 tensor([[-0.4148,  0.0722,  0.0533,  0.4390]], requires_grad=True),
 Parameter containing:
 tensor([-0.3543], requires_grad=True)]

In [29]:
# Eval algorithm
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

# Fitting algorithm
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

# Check the initial value that val_loss have
result = evaluate(model, val_loader)
print(result)

{'val_loss': 1140.5899658203125}


In [30]:
# Start with the Fitting
epochs = 90
lr = 1e-8
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 768.5389
Epoch [40], val_loss: 491.7601
Epoch [60], val_loss: 381.5482
Epoch [80], val_loss: 385.4811
Epoch [90], val_loss: 391.4060


In [31]:
# Train repeatdly until have a 'good' val_loss
epochs = 20
lr = 1e-9
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 392.6694


In [32]:
# Prediction Algorithm
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

# Testing the model with some samples
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([2.1158e+03, 1.3600e+01, 4.0324e+04, 0.0000e+00])
Target: tensor([9.8600])
Prediction: tensor([-165.3362])


In [33]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([2.1168e+03, 7.8700e+00, 2.4524e+04, 0.0000e+00])
Target: tensor([6.7860])
Prediction: tensor([-444.8370])
